# HACKtheMACHINE 2021 | Track 2: Data Science, Detective Bot 
# Starter Notebook 1: Processed Data

## Background

Track 2, the Data Science track, is split into three challenges and uses real-world cyber data: benign and malicious binary files used in actual cyber testing. The Navy wants to learn how well Machine Learning and Artificial Intelligence (ML/AI) can detect and identify malicious cyber activity with high classification accuracy and performance. Participants will be provided with the raw data, plus simplified data sets and starter notebook to expedite initial solutions. 

## How to use this notebook

This notebook will provide techniques for participants to get started looking at the data quickly. We highly suggest researching portable executable (PE) files as well as the Elastic Malware Benchmark for Empowering Researchers (EMBER), an open-source feature extraction tool that we will be leveraging in this notebook. Please refer to the `Recommended Resources` below. 

## Recommended Resources

### Python Tutorials:
* [Python Fundamentals](https://www.tutorialspoint.com/python/index.htm)

### Python Libraries:
* [Pandas](https://pandas.pydata.org/docs/) - Data analysis and manipulation 
* [Matplotlib](https://matplotlib.org/stable/contents.html) - Data visualizations and plots
* [EMBER - Elastic Malware Benchmark for Empowering Researchers](https://github.com/elastic/ember)
*[Sklearn](https://scikit-learn.org/stable/index.html) - Machine Learning in Python

### Papers/Articles:
* [EMBER: An Open Dataset for Training Static PE Malware Machine Learning Models](https://arxiv.org/pdf/1804.04637.pdf) - Original Paper for the EMBER library
* [The rise of machine learning for detection and classification of malware: Research developments, trends, and challenges](https://reader.elsevier.com/reader/sd/pii/S1084804519303868?token=1B85475195D5551845E9E32F74B919C68B83EA9FA742D8101974DAB0C602DE47AF793D75C2ED033A60E68D54DB1F7998&originRegion=us-east-1&originCreation=20211112194210)
* [A brief introduction to PE format](https://medium.com/ax1al/a-brief-introduction-to-pe-format-6052914cc8dd)
* Refer to `PE Files Explained.docx` document provided

## Data Dictionary

The following data dictionary describes the columns or fields of the data set and a description of the objects. This information with more detail can also be found in the `EMBER` documentation of the `features.py` file at: https://github.com/elastic/ember/blob/master/ember/features.py 

| Field Name | Description | 
|------------|-------------|
| sha256 | The Secure Hash Algorithm (SHA) is a cryptographic hash function like a signature or fingerprints for a data set. Even if one symbol is changed the algorithm will produce a different hash value. The SHA256 algorithm generates a fixed size 256-bit (32-byte) hash. The SHA256 algorithm is used to ensure you acquire the same data as the original. For exmaple, if you download something you can check if the data has not changed (due to network errors or malware injection) by comparing the hashes of your file and the original.|
| histogram | Byte histogram (count + non-normalized) over the entire binary file. The byte histogram contains 256 integer values and represent the counts of each byte value within the value. When generating model features the byte histgoram is normalized to a distribution, since file size is represented as a feature in the general file information. | 
| byteentropy | 2D byte/entropy histogram based loosely on (Saxe and Berlin, 2015). This roughly approximates the joint probability of byte value and local entropy. See Section 2.1.1 in https://arxiv.org/pdf/1508.03096.pdf for more info. The byte entropy histogram approximates the joint distriubtion p(H, X) of entropy H and byte value X. By computing the scalar entropy H for a fixed-length window and pairing it with each byte occurrence within the window. This is repeated as the window slides across the input bytes. |
| strings | Contains simple statistics about printable strings of the following: <ul><li>`numstrings`: number of strings <li> `avlength`: average length of strings <li>`printabledist`: histogram of the printable characters within those strings <li>`printables`: distinct information from byte histogram information from the byte histogram information since its derived only from strings containing at least 5 consecutive printable characters <li>`entropy`: entropy of characters across all printable strings <li>`paths` number of strings that begin with **C:** (case insensitive) that may indicate a path <li>`urls`: the number of occurences of **http://** or **https://** (case insensitive) that may indicate a URL <li>`registry`: number of occurrences of HKEY that may indicate a registry key, <li>`MZ`: number of occurrences of the short string MZ |
| general | Provides general file information. 0/1 indicates a binary output <ul><li>`size`: length of bytes <li>`vsize`: virtual size <li>`has_debug`: 0/1  <li>`exports`: 0/1 <li>`imports`: 0/1 <li>`has_relocations`: 0/1  <li>`has_resources`: 0/1 <li>`has_signature`: 0/1 <li>`has_tls`: 0/1 <li>`symbols`: 0/1 |
| header | Provides header file information on machine, architecture, OS, link and other information: <ul><li> `coeff`: [ `timestamp`, `machine`,`characteristics` ] <li> `optional`: [`subsystem`, `dll_characteristics`, `magic`, `major_image_version`, `minor_linker_version`, `major_operating_system_version`, `minor_operating_system_version`, `major_subsystem_version`, `minor_subsystem_version`, `sizeof_code`, `sizeof_headers`, `sizeof_heap_commit`]
| section | Information about section names, sizes and entropy. Uses hashing trick to summarize all this section into a feature vector. <ul><li> `imports`: [`KERNEL32.dll` : [`GetTickCount`] | 
| imports | Information about imported libraries and functions from the import address table. Note that the total number of imported functions is contained in GeneralFileInfo. |
| exports | Information about exported functions. Note that the total number of exported functions is contained in GeneralFileInfo.|
| datadirectories | Extracts size and virtual address of the first 15 data dictectories. |
| label / category | Class label indicating benign `0` or malicious `1`|

## Load Libraries

In [ ]:
# install libraries
!pip install pandas
!pip install sklearn
!pip install matplotlib

In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

## Load and Read Files

We will use the `.read_excel()` function to read datasets into the pandas dataframe. 

Loading the data may take a while, so please wait for the data to load. The `[*]` symbol on the left side of the code block indicates the cell is still running.

Note: To read in flat files of `flatten_train.xlsx` and raw files `raw_train.xlsx`, place them in the same directory as where the notebook is executed. If the data is in another folder you will have to update the file path in the parentheses after `.read_excel()`.

In [3]:
# Flattened EMBER Feature set
# Easier to feed into ML models right away
df1 = pd.read_excel("../Data/flatten_train.xlsx")

# Raw EMBER features
# More data but will require more preprocessing
df2 = pd.read_excel("../Data/raw_train.xlsx")

## Exploratory Data Analysis 

In the next part, we are going to look at DataFrame 1 and DataFrame 2. For your solution you can work with either DataFrame or combine both into one large DataFrame. (They are split into 2, due to the dimensionality of the dataset.) 


It may be useful to do feature selection to reduce the set of features to a meaningful set
for your modeling problem.

Note: Each DataFrame has a column named `category` which idenfies the file as:
* `0` - Benign
* `1` - Malware



## Let's take a look at the first dataframe
Let's take a look at the flattened dataset in a pandas dataframe. We can look at data types, shape of data, null counts.

### DataFrame 1

In [4]:
df1.head()

,sha256,histogram_0,histogram_1,histogram_2,histogram_3,histogram_4,histogram_5,histogram_6,histogram_7,histogram_8,...,datadirectories_13_name,datadirectories_13_size,datadirectories_13_virtual_address,datadirectories_14_name,datadirectories_14_size,datadirectories_14_virtual_address,datadirectories_15_name,datadirectories_15_size,datadirectories_15_virtual_address,category
0,15f88c44e29e74965cbec984cd4244786eeb4ccde02328...,127242,9185,3818,3127,13300,1594,1781,1493,9938,...,DELAY_IMPORT_DESCRIPTOR,0,0,CLR_RUNTIME_HEADER,0,0,???,0,0,0
1,29bb2a854de0f99bdb7120870ad42af6a3b5b5c6fc3578...,36152,2617,4120,876,1711,755,672,521,2278,...,DELAY_IMPORT_DESCRIPTOR,0,0,CLR_RUNTIME_HEADER,0,0,???,0,0,0
2,5cd8667d8375bf799fe89ac9b74cf08bf69dfa18a4e551...,39986,7112,7953,6604,6542,6441,5869,6422,6224,...,DELAY_IMPORT_DESCRIPTOR,0,0,CLR_RUNTIME_HEADER,0,0,???,0,0,0
3,ce63706c0adf4851a8d2eb26fa97750cfaec7345151367...,253873,32714,22629,17500,15355,6691,10881,4258,11992,...,DELAY_IMPORT_DESCRIPTOR,0,0,CLR_RUNTIME_HEADER,0,0,???,0,0,0
4,5e65dd9986fc47c5fe45208fab5e1c78730699800eaf96...,12464,1020,502,264,1918,218,262,91,846,...,DELAY_IMPORT_DESCRIPTOR,0,0,CLR_RUNTIME_HEADER,0,0,???,0,0,0


### Size of Data

Now, we are interested to see how much data is in the dataframe:

`.shape` is an attribute of a dataframe that returns a tuple of (number of rows, number of columns) in the dataframe. Because shape is not a function we do not need the parentheses at the end.

In [5]:
df1.shape

(18000, 714)

### Data Types 

`.dtypes` is an attribute that returns a list of each column and what type each columns contents is. Again this is not a function so we do not put parentheses at the end.

In [6]:
#df1.dtypes # prints summary only
df1.info(verbose = True) # prints entire list of datatypes 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 714 columns):
 #    Column                                                     Dtype  
---   ------                                                     -----  
 0    sha256                                                     object 
 1    histogram_0                                                int64  
 2    histogram_1                                                int64  
 3    histogram_2                                                int64  
 4    histogram_3                                                int64  
 5    histogram_4                                                int64  
 6    histogram_5                                                int64  
 7    histogram_6                                                int64  
 8    histogram_7                                                int64  
 9    histogram_8                                                int64  
 10   histogra

### Null Values in the Data 

The `.isnull()` function takes a dataframe and indicates whether values are missing for each value.

In [7]:
df1.isnull()

,sha256,histogram_0,histogram_1,histogram_2,histogram_3,histogram_4,histogram_5,histogram_6,histogram_7,histogram_8,...,datadirectories_13_name,datadirectories_13_size,datadirectories_13_virtual_address,datadirectories_14_name,datadirectories_14_size,datadirectories_14_virtual_address,datadirectories_15_name,datadirectories_15_size,datadirectories_15_virtual_address,category
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
df1.isnull().sum()

sha256                                0
histogram_0                           0
histogram_1                           0
histogram_2                           0
histogram_3                           0
                                     ..
datadirectories_14_virtual_address    0
datadirectories_15_name               0
datadirectories_15_size               0
datadirectories_15_virtual_address    0
category                              0
Length: 714, dtype: int64

## Now let's take a look at the second data frame.

### DataFrame 2

Let's take a look at the preprocessed data in a pandas dataframe of the second dataset. We can look at data types, shape of data, null counts.

In [9]:
df2.head()

,sha256,histogram,byteentropy,strings,general,header,section,imports,exports,datadirectories,category
0,15f88c44e29e74965cbec984cd4244786eeb4ccde02328...,"[127242, 9185, 3818, 3127, 13300, 1594, 1781, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'numstrings': 4015, 'avlength': 20.6953922789...","{'size': 683022, 'vsize': 11714560, 'has_debug...","{'coff': {'timestamp': 1323701732, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...","{'cygwin1.dll': ['__assert_func', '__errno', '...",[],"[{'name': 'EXPORT_TABLE', 'size': 0, 'virtual_...",0
1,29bb2a854de0f99bdb7120870ad42af6a3b5b5c6fc3578...,"[36152, 2617, 4120, 876, 1711, 755, 672, 521, ...","[14324, 0, 0, 6, 1, 1, 0, 0, 0, 0, 1, 3, 0, 0,...","{'numstrings': 808, 'avlength': 12.23391089108...","{'size': 176128, 'vsize': 180224, 'has_debug':...","{'coff': {'timestamp': 1141554791, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...",{'aModelLib.dll': ['?CreateAnotherModel@@YAPAU...,['GetPlugIn'],"[{'name': 'EXPORT_TABLE', 'size': 73, 'virtual...",0
2,5cd8667d8375bf799fe89ac9b74cf08bf69dfa18a4e551...,"[39986, 7112, 7953, 6604, 6542, 6441, 5869, 64...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'numstrings': 6967, 'avlength': 6.13248169944...","{'size': 1513875, 'vsize': 180224, 'has_debug'...","{'coff': {'timestamp': 1300357374, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...","{'oleaut32.dll': ['SysFreeString', 'SysReAlloc...",[],"[{'name': 'EXPORT_TABLE', 'size': 0, 'virtual_...",0
3,ce63706c0adf4851a8d2eb26fa97750cfaec7345151367...,"[253873, 32714, 22629, 17500, 15355, 6691, 108...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'numstrings': 8836, 'avlength': 11.7158216387...","{'size': 927852, 'vsize': 921600, 'has_debug':...","{'coff': {'timestamp': 1403657733, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...","{'libogg-0.dll': ['oggpackB_bytes', 'oggpackB_...","['theora_encode_init', 'theora_encode_YUVin', ...","[{'name': 'EXPORT_TABLE', 'size': 470, 'virtua...",0
4,5e65dd9986fc47c5fe45208fab5e1c78730699800eaf96...,"[12464, 1020, 502, 264, 1918, 218, 262, 91, 84...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'numstrings': 557, 'avlength': 16.80610412926...","{'size': 83026, 'vsize': 98304, 'has_debug': 0...","{'coff': {'timestamp': 1252253596, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...","{'libcairo-2.dll': ['cairo_arc', 'cairo_clip',...","['g_module_check_init', 'theme_create_rc_style...","[{'name': 'EXPORT_TABLE', 'size': 157, 'virtua...",0


### Size of Data

Now, we are interested to see how much data is in the dataframe:

`.shape` is an attribute of a dataframe that returns a tuple of (number of rows, number of columns) in the dataframe. Because shape is not a function we do not need the parentheses at the end.

In [10]:
df2.shape

(18000, 11)

### Data Types

`.dtypes` is an attribute that returns a list of each column and what type each columns contents is. Again this is not a function so we do not put parentheses at the end.

In [11]:
df2.dtypes #prints summary only
df2.info(verbose = True) #prints entire list of datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sha256           18000 non-null  object
 1   histogram        18000 non-null  object
 2   byteentropy      18000 non-null  object
 3   strings          18000 non-null  object
 4   general          18000 non-null  object
 5   header           18000 non-null  object
 6   section          18000 non-null  object
 7   imports          18000 non-null  object
 8   exports          18000 non-null  object
 9   datadirectories  18000 non-null  object
 10  category         18000 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1.5+ MB


### Null Values in the Data

The `.isnull()` function takes a dataframe and indicates whether values are missing for each value.

In [12]:
df2.isnull()

,sha256,histogram,byteentropy,strings,general,header,section,imports,exports,datadirectories,category
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
17995,False,False,False,False,False,False,False,False,False,False,False
17996,False,False,False,False,False,False,False,False,False,False,False
17997,False,False,False,False,False,False,False,False,False,False,False
17998,False,False,False,False,False,False,False,False,False,False,False


In [13]:
df2.isnull().sum()

sha256             0
histogram          0
byteentropy        0
strings            0
general            0
header             0
section            0
imports            0
exports            0
datadirectories    0
category           0
dtype: int64